<a href="https://colab.research.google.com/github/richardjana/freeCodeCamp/blob/main/Machine_Learning_with_Python/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# prepare data: convert categorical data, scale numerical data, test / train split
for col in ['sex', 'smoker']:
  category_map = {val: i for i, val in enumerate(dataset[col].unique())}
  dataset[col] = dataset[col].map(category_map).astype(int)

category_map = {'southwest': (0, 1),
                'southeast': (0, 0),
                'northwest': (1, 1),
                'northeast': (1, 0)}
dataset[['x', 'y']] = dataset['region'].map(category_map).apply(pd.Series)
dataset = dataset.drop(columns=['region'])

scaler = MinMaxScaler()
numerical_cols = ['age', 'bmi']
dataset[numerical_cols] = scaler.fit_transform(dataset[numerical_cols])

train, test_dataset = train_test_split(dataset, test_size=0.20, random_state=42, shuffle=True)

train_labels = train.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
import matplotlib.pyplot as plt

def plot_train_history(history):
  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.plot(history.history['loss'], label='Training Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss (MSE)')
  plt.title('Training and Validation Loss')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.plot(history.history['mae'], label='Training MAE')
  plt.xlabel('Epoch')
  plt.ylabel('MAE')
  plt.title('Training and Validation MAE')
  plt.legend()

  plt.show()

In [ ]:
model = keras.Sequential([layers.Dense(128, activation='relu', input_shape=(7,)),
                          layers.Dense(128, activation='relu'),
                          layers.Dense(128, activation='relu'),
                          layers.Dense(128, activation='relu'),
                          layers.Dense(1)
                          ])

optimizer = Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mae', 'mse'])

history = model.fit(train, train_labels, epochs=100, verbose=1)

plot_train_history(history)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
